In [ ]:
import os, sys
# module_path = os.path.abspath(os.path.join('../../Prompt-based-class-incremental-learning-in-Federated-Learning/CODA_Prompt_CVPR2023_Code/'))
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import torch, torchvision, numpy as np
from dataloaders import dataloader
from dataloaders.dataloader import iCIFAR100_Fed
import random
import dataloaders

In [ ]:
seed = 2023
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
# !pip install fairscale
from models.vit_coda_p import vit_pt_imnet
import models
from models import  vit_coda_p


In [ ]:
prompt_params = [10, [100, 8, 0 ,0 ,0]]

dual_prompt_params  = [10, [10, 20, 6]]

In [ ]:
model_codap = vit_pt_imnet(out_dim=100, prompt_flag = 'codap', prompt_param=prompt_params)
# model_dual = vit_pt_imnet(out_dim=100, prompt_flag = 'dual', prompt_param=dual_prompt_params)
# model_l2p = vit_coda_p.vit_pt_imnet(out_dim=100, prompt_flag = 'l2p', prompt_param=prompt_params)
# model_base = vit_pt_imnet(out_dim=100, prompt_flag = "None", prompt_param=None)

In [ ]:
model_codap = torch.nn.DataParallel(model_codap)
model_codap.cuda()
# for key, param in model_codap.named_parameters():
#     if 'module.prompt.e_p_0' in key: 
#         print(key, param)

In [ ]:
# load data
tasks = []
tasks_logits = []
p = 0
max_task = -1
other_split_size = 10 
first_split_size = 10 
num_classes = 100 
class_order = np.arange(num_classes).tolist()
class_order_logits = np.arange(num_classes).tolist()
rand_split = True

if seed > 0 and rand_split:
    print('=============================================')
    print('Shuffling....')
    print('pre-shuffle:' + str(class_order))
    random.seed(seed)
    random.shuffle(class_order)
    print('post-shuffle:' + str(class_order))
    print('=============================================')
            
            
while p < num_classes and (max_task == -1 or len(tasks) < max_task):
    inc = other_split_size if p > 0 else first_split_size
    tasks.append(class_order[p:p + inc])
    tasks_logits.append(class_order_logits[p:p + inc])
    p += inc
num_tasks = len(tasks)
task_names = [str(i + 1) for i in range(num_tasks)]
train_transform = dataloaders.utils.get_transform(dataset='CIFAR100_Fed', phase='train', aug=True,
                                                          resize_imnet=True)

# train = iCIFAR100_Fed(root='./data', train=True, transform=torchvision.transforms.ToTensor())
train_dataset = dataloader.iCIFAR100_Fed(root= "data", train = True, transform=train_transform,
                                         num_clients = 10,
                                         iid  = 0, 
                                        download_Flag = True,
                                        validation=False,
                                         tasks=tasks, seed=0
                                        )


In [ ]:
st1 = torch.load("/data/gaurav/Projects/promptFCL/_outputs/ICCV/CIFAR100/10-task/fedMoon1/vit/coda-iid_fedMoon_1/models/repeat-1/task-1/class.pth")
st2 = torch.load("/data/gaurav/Projects/promptFCL/_outputs/ICCV/CIFAR100/10-task/fedMoon1/vit/coda-iid_fedMoon_1/models/repeat-1/task-2/class.pth")
st3 = torch.load("/data/gaurav/Projects/promptFCL/_outputs/ICCV/CIFAR100/10-task/fedMoon1/vit/coda-iid_fedMoon_1/models/repeat-1/task-3/class.pth")


In [ ]:
import copy
model1 = copy.deepcopy(model_codap)
model2 = copy.deepcopy(model_codap)
model3 = copy.deepcopy(model_codap)
# model1 = torch.nn.DataParallel(model1)
# model2 = torch.nn.DataParallel(model2)
# model1.cuda(), model2.cuda()
model1.cuda()
model2.cuda()
model3.cuda()

model1.load_state_dict(st1)
model2.load_state_dict(st2)
model3.load_state_dict(st3)


In [ ]:
class TripletLoss(torch.nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        distance_positive = (anchor - positive).pow(2).sum(1)
        distance_negative = (anchor - negative).pow(2).sum(1)
        losses = torch.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()

In [ ]:
# img train_dataset[0][0]
img = train_dataset.data[0]
img.shape

In [ ]:
train_dataset.load_dataset(t=2, train=True,client=2)
trian_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)

In [ ]:
# train_loader1 = next(iter(trian_loader))



In [ ]:
# imgs = train_loader1[0].cuda()
# labels = train_loader1[1].cuda()
# imgs.shape, labels.shape


In [ ]:
optimizer = torch.optim.Adam(model2.parameters(), lr=0.0001)
from tqdm import tqdm
# ce loss
criterion = torch.nn.CrossEntropyLoss()
model_base = model2 
# use tqdm for progress bar
for epoch in tqdm(range(10)):
    # accuracy
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(trian_loader):
        data, target = data.cuda(), target.cuda()
        a1, _, anchor = model2(data, train = True)
        with torch.no_grad():
            a2, _, positive = model_base(data, train = True)
            a3, _, negative = model1(data, train = True)
        loss = TripletLoss()(anchor, positive, negative)
        ce_loss = criterion(a1, target)
        total_loss = loss + ce_loss

        # accuracy
        _, predicted = torch.max(a1.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

        

        # 
        # tqdm.write(str(epoch) + '/' + str(10) + str(batch_idx) + str(loss))
        # print(epoch , '/', 10 , batch_idx, loss)
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
    print(epoch, loss, ce_loss, total_loss)
    print('Accuracy: %.2f %%' % (100 * correct / total))
    print('=============================================')




In [ ]:
model2.state_dict()['module.prompt.e_p_0']

In [ ]:
st2['module.prompt.e_p_0']

In [ ]:
len(anchor), len(positive), len(negative), len(a1), len(a2), len(a3)

In [ ]:
c2l = TripletLoss() 
# for i in range(128):
#     loss = c2l(anchor[i], positive[i], negative[i])
#     print(loss)
loss = c2l(e1, e2, e3)
loss.mean()

In [ ]:
e1 = st1['module.prompt.e_p_0']
e2 = st2['module.prompt.e_p_0']
e3 = st3['module.prompt.e_p_0']
e1.shape, e2.shape, e3.shape

In [ ]:
# e1-e2 , e1-e3

In [ ]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
cos((e1 -e2), (e1- e3)).mean()

In [ ]:
posi = cos(logits1, logits3)
nega = cos(logits1, logits2)
numerator = torch.exp(posi/tau)
denominator = torch.exp(posi/tau) + torch.exp(nega/tau)
fedmoonLoss = -torch.log(numerator/denominator)

In [ ]:
fedmoonLoss

In [ ]:
model_codap = torch.nn.DataParallel(model_codap)
model_codap = model_codap.cuda()

In [ ]:
import copy
m1 = copy.deepcopy(model_codap) #.load_state_dict(st1)
m2 = copy.deepcopy(model_codap) #.load_state_dict(st2)

In [ ]:
m1.load_state_dict(st1)
m2.load_state_dict(st2)

In [ ]:
for i in range(100):
     a= model2.state_dict()['module.prompt.e_p_0'][i] - model1.state_dict()['module.prompt.e_p_0'][i]
     print( i , torch.sum(a))
     

In [ ]:
# model2.state_dict()['module.prompt.e_p_0']